In [35]:
import numpy as np
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random

from efficientnet_pytorch import EfficientNet

model_name = 'efficientnet-b0' 
image_size = EfficientNet.get_image_size(model_name)
print(image_size)
model = EfficientNet.from_pretrained(model_name, num_classes=50)

224
Loaded pretrained weights for efficientnet-b0


In [36]:
## 데이터 로드
batch_size  = 128
random_seed = 555
random.seed(random_seed)
torch.manual_seed(random_seed)

In [37]:
## make dataset
from torchvision import transforms, datasets
data_path = '../../data/ic/images'  # class 별 폴더로 나누어진걸 확 가져와서 라벨도 달아준다
ic_dataset = datasets.ImageFolder(
                                data_path,
                                transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ]))

In [38]:
## data split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
train_idx, tmp_idx = train_test_split(list(range(len(ic_dataset))), test_size=0.2, random_state=random_seed)
datasets = {}
datasets['train'] = Subset(ic_dataset, train_idx)
tmp_dataset       = Subset(ic_dataset, tmp_idx)

val_idx, test_idx = train_test_split(list(range(len(tmp_dataset))), test_size=0.5, random_state=random_seed)
datasets['valid'] = Subset(tmp_dataset, val_idx)
datasets['test']  = Subset(tmp_dataset, test_idx)

In [41]:
## data loader 선언
dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                              batch_size=batch_size, shuffle=True,
                                              num_workers=0) # window 0, 원래 4
dataloaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=0) # window 0, 원래 4
dataloaders['test']  = torch.utils.data.DataLoader(datasets['test'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=0) # window 0, 원래 4
batch_num['train'], batch_num['valid'], batch_num['test'] = len(dataloaders['train']), len(dataloaders['valid']), len(dataloaders['test'])
print('batch_size : %d,  tvt : %d / %d / %d' % (batch_size, batch_num['train'], batch_num['valid'], batch_num['test']))

batch_size : 128,  tvt : 32 / 4 / 4


In [42]:
## data check
import torchvision
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

num_show_img = 50

class_names = {
    "0": "apple_pie",      
    "1": "baby_back_ribs",   
    "2": "baklava",   
    "3": "beef_carpaccio", 
    "4": "beef_tartare",   
    "5": "beet_salad",   
    "6": "beignets",  
    "7": "bibimbap",      
    "8": "bread_pudding",   
    "9": "breakfast_burrito",   

    "10": "bruschetta", 
    "11": "caesar_salad",   
    "12": "cannoli",   
    "13": "caprese_salad",  
    "14": "carrot_cake",      
    "15": "ceviche",   
    "16": "cheese_plate",   
    "17": "cheesecake", 
    "18": "chicken_curry",   
    "19": "chicken_quesadilla",   

    "20": "chicken_wings",  
    "21": "chocolate_cake",      
    "22": "chocolate_mousse",   
    "23": "churros",   
    "24": "clam_chowder", 
    "25": "club_sandwich",   
    "26": "crab_cakes",   
    "27": "creme_brulee",  
    "28": "croque_madame",      
    "29": "cup_cakes",   

    "30": "deviled_eggs",   
    "31": "donuts", 
    "32": "dumplings",   
    "33": "edamame",   
    "34": "eggs_benedict",  
    "35": "escargots",      
    "36": "falafel",   
    "37": "filet_mignon",   
    "38": "fish_and_chips", 
    "39": "foie_gras",   

    "40": "french_fries",   
    "41": "french_onion_soup",   
    "42": "french_toast", 
    "43": "fried_calamari",   
    "44": "fried_rice",   
    "45": "frozen_yogurt",   
    "46": "garlic_bread", 
    "47": "gnocchi",   
    "48": "greek_salad",   
    "49": "grilled_cheese_sandwich"
}

# train check
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:num_show_img])  # batch의 이미지를 오려부친다
imshow(out, title=[class_names[str(int(x))] for x in classes[:num_show_img]])
# valid check
inputs, classes = next(iter(dataloaders['valid']))
out = torchvision.utils.make_grid(inputs[:num_show_img])  # batch의 이미지를 오려부친다
imshow(out, title=[class_names[str(int(x))] for x in classes[:num_show_img]])
# test check
inputs, classes = next(iter(dataloaders['test']))
out = torchvision.utils.make_grid(inputs[:num_show_img])  # batch의 이미지를 오려부친다
imshow(out, title=[class_names[str(int(x))] for x in classes[:num_show_img]])

MemoryError: Unable to allocate 87.5 MiB for an array with shape (1584, 1810, 4) and data type float64